In [2]:
import os
os.environ["MANTLE"] = "coreir"
import silica as si
import magma as m
from silica import bits, add, uint, list
from magma.testing.coroutine import check

@si.coroutine(inputs={"wdata": si.Bits(4), "wen": si.Bit, "ren": si.Bit})
def Fifo():
    memory = list(bits(0, 4) for i in range(4))
    raddr = uint(0, 2)
    waddr = uint(0, 2)
    next_empty = True
    next_full  = False
    wdata, wen, ren = yield
    while True:
        full = next_full
        empty = next_empty
        rdata = memory[raddr]
        if wen and not full:
            memory[waddr] = wdata
            waddr = waddr + 1
            next_full = raddr == waddr
            next_empty = False
        if ren and not empty:
            raddr = raddr + 1
            next_empty = raddr == waddr
            next_full = False
        wdata, wen, ren = yield rdata, empty, full
        
fifo = si.compile(Fifo(), file_name="build/silica_fifo.py")
with open("build/silica_fifo.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                            wire(and_.I1[j], io.S[i])
                    else:
   

In [3]:
expected_trace = [
        {'wdata': 1, 'wen': 0, 'ren': 1, 'rdata': 0, 'full': False, 'empty': True, 'memory': [0, 0, 0, 0], 'raddr': 0, 'waddr': 0},
        {'wdata': 2, 'wen': 1, 'ren': 0, 'rdata': 0, 'full': False, 'empty': True, 'memory': [2, 0, 0, 0], 'raddr': 0, 'waddr': 1},
        {'wdata': 3, 'wen': 1, 'ren': 1, 'rdata': 2, 'full': False, 'empty': False, 'memory': [2, 3, 0, 0], 'raddr': 1, 'waddr': 2},
        {'wdata': 4, 'wen': 1, 'ren': 0, 'rdata': 3, 'full': False, 'empty': False, 'memory': [2, 3, 4, 0], 'raddr': 1, 'waddr': 3},
        {'wdata': 5, 'wen': 0, 'ren': 1, 'rdata': 3, 'full': False, 'empty': False, 'memory': [2, 3, 4, 0], 'raddr': 2, 'waddr': 3},
        {'wdata': 6, 'wen': 0, 'ren': 1, 'rdata': 4, 'full': False, 'empty': False, 'memory': [2, 3, 4, 0], 'raddr': 3, 'waddr': 3},
        {'wdata': 7, 'wen': 1, 'ren': 0, 'rdata': 0, 'full': False, 'empty': True, 'memory': [2, 3, 4, 7], 'raddr': 3, 'waddr': 0},
        {'wdata': 8, 'wen': 0, 'ren': 1, 'rdata': 7, 'full': False, 'empty': False, 'memory': [2, 3, 4, 7], 'raddr': 0, 'waddr': 0},
        {'wdata': 9, 'wen': 1, 'ren': 1, 'rdata': 2, 'full': False, 'empty': True, 'memory': [9, 3, 4, 7], 'raddr': 0, 'waddr': 1},
        {'wdata': 10, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': False, 'empty': False, 'memory': [9, 10, 4, 7], 'raddr': 0, 'waddr': 2},
        {'wdata': 11, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': False, 'empty': False, 'memory': [9, 10, 11, 7], 'raddr': 0, 'waddr': 3},
        {'wdata': 12, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': False, 'empty': False, 'memory': [9, 10, 11, 12], 'raddr': 0, 'waddr': 0},
        {'wdata': 13, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': True, 'empty': False, 'memory': [9, 10, 11, 12], 'raddr': 0, 'waddr': 0},
        {'wdata': 13, 'wen': 0, 'ren': 1, 'rdata': 9, 'full': True, 'empty': False, 'memory': [9, 10, 11, 12], 'raddr': 1, 'waddr': 0},
        {'wdata': 14, 'wen': 1, 'ren': 1, 'rdata': 10, 'full': False, 'empty': False, 'memory': [14, 10, 11, 12], 'raddr': 2, 'waddr': 1},
]

sim_fifo = Fifo()
inputs = ("wdata", "wen", "ren")
outputs = ("rdata", "full", "empty")
states = ("memory", "raddr", "waddr")
for i, trace in enumerate(expected_trace):
    args = ()
    for input_ in inputs:
        args += (trace[input_], )
    sim_fifo.send(args)
    for output in outputs:
        assert getattr(sim_fifo, output) == trace[output], (i, output,  getattr(sim_fifo, output), trace[output])
    for state in states:
        assert getattr(sim_fifo, state) == trace[state], (i, state)

@si.coroutine
def inputs_generator(N):
    while True:
        for trace in expected_trace:
            wdata = bits(trace["wdata"], N)
            wen = bool(trace["wen"])
            ren = bool(trace["ren"])
            yield wdata, wen, ren
        
check(fifo, Fifo(), len(expected_trace), inputs_generator(4))
print("Passed!")

Passed!


In [7]:
!magma -o coreir -m coreir -t Fifo build/silica_fifo.py
!coreir -i build/silica_fifo.json -o build/silica_fifo.v
# !magma -o verilog -m lattice -t Fifo build/silica_fifo.py
!yosys -p 'synth_ice40 -top Fifo -blif build/silica_fifo.blif' build/silica_fifo.v | grep -A 20 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_fifo.txt build/silica_fifo.blif
!icetime -tmd hx1k build/silica_fifo.txt  | grep -B 2 "Total path delay"

In Run Generators
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!

Modified?: Yes
2.27. Printing statistics.

=== Fifo ===

   Number of wires:               2395
   Number of wire bits:           4223
   Number of public wires:        2371
   Number of public wire bits:    4199
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 74
     SB_DFF                         24
     SB_LUT4                        50

2.28. Executi

In [5]:
!yosys -p 'synth_ice40 -top fifo -blif build/verilog_fifo.blif' ../verilog/fifo.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/verilog_fifo.txt build/verilog_fifo.blif
!icetime -tmd hx1k build/verilog_fifo.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== fifo ===

   Number of wires:                 22
   Number of wire bits:             46
   Number of public wires:          12
   Number of public wire bits:      28
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 29
     SB_CARRY                        2
     SB_DFFE                        11
     SB_LUT4                        16
seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/verilog_fifo.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          15 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          25 / 1280
  DFF        11
  CARRY      4
  CARRY, DFF 0
  DFF PASS   5
  CARRY PASS 2
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted CLK$2, 11 

In [6]:
import magma
from magma.testing.verilator import compile as compileverilator
from magma.testing.verilator import run_verilator_test
from magma.bit_vector import BitVector

def check_verilog(circ, circ_sim, num_cycles, inputs_generator=None):
    test_vectors = magma.testing.coroutine.testvectors(circ, circ_sim, num_cycles,
            inputs_generator if inputs_generator else None)

    name = "fifo"
    with open(f"build/sim_{name}_verilator.cpp", "w") as verilator_harness:
        verilator_harness.write(f'''\
#include "V{name}.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

void check(const char* port, int a, int b, int cycle) {{
    if (!(a == b)) {{
        std::cerr << \"Got      : \" << a << std::endl;
        std::cerr << \"Expected : \" << b << std::endl;
        std::cerr << \"Cycle    : \" << cycle << std::endl;
        std::cerr << \"Port     : \" << port << std::endl;
        exit(1);
    }}
}}

int main(int argc, char **argv, char **env) {{
    Verilated::commandArgs(argc, argv);
    V{name}* top = new V{name};
''')
    
        cycle = 0
        for inputs, outputs in zip(test_vectors[0], test_vectors[1]):
            print(cycle)
            print(inputs) 
            print(outputs)
            for port_name, value in inputs.items():
                if isinstance(value, list):
                    string = ""
                    for elem in value:
                        string = elem.as_binary_string()[2:] + string
                    value = "0b" + string
                elif isinstance(value, BitVector):
                    value = value.as_int()
                else:
                    value = int(value)
                verilator_harness.write("    top->{} = {};\n".format(port_name, value))
            for port_name, value in outputs.items():
                if isinstance(value, BitVector):
                    value = value.as_binary_string()
                else:
                    value = int(value)
                verilator_harness.write("    top->eval();\n")
                verilator_harness.write("    check(\"{port_name}\", top->{port_name}, {expected}, {cycle});\n".format(port_name=port_name, expected=value, cycle=cycle))

            verilator_harness.write("    top->CLK = 0;\n")
            verilator_harness.write("    top->eval();\n")
            verilator_harness.write("    top->CLK = 1;\n")
            verilator_harness.write("    top->eval();\n")
            cycle += 1
        verilator_harness.write("}\n")

    run_verilator_test(
        "fifo",
        f"sim_{name}_verilator",
        name,
        "-I../../verilog"
    )

check_verilog(fifo, Fifo(), len(expected_trace), inputs_generator(4))
print("Passed!")

0
{'wdata': bits(1, 4), 'wen': False, 'ren': True}
{'rdata': bits(0, 4), 'full': False, 'empty': True}
1
{'wdata': bits(2, 4), 'wen': True, 'ren': False}
{'rdata': bits(0, 4), 'full': False, 'empty': True}
2
{'wdata': bits(3, 4), 'wen': True, 'ren': True}
{'rdata': bits(2, 4), 'full': BitVector(False, 1), 'empty': False}
3
{'wdata': bits(4, 4), 'wen': True, 'ren': False}
{'rdata': bits(3, 4), 'full': False, 'empty': BitVector(False, 1)}
4
{'wdata': bits(5, 4), 'wen': False, 'ren': True}
{'rdata': bits(3, 4), 'full': BitVector(False, 1), 'empty': False}
5
{'wdata': bits(6, 4), 'wen': False, 'ren': True}
{'rdata': bits(4, 4), 'full': False, 'empty': BitVector(False, 1)}
6
{'wdata': bits(7, 4), 'wen': True, 'ren': False}
{'rdata': bits(0, 4), 'full': False, 'empty': BitVector(True, 1)}
7
{'wdata': bits(8, 4), 'wen': False, 'ren': True}
{'rdata': bits(7, 4), 'full': BitVector(False, 1), 'empty': False}
8
{'wdata': bits(9, 4), 'wen': True, 'ren': True}
{'rdata': bits(2, 4), 'full': False, '